# Example: Heterogeneous Inflow for multiple conditions

In [ ]:
"""Example: Heterogeneous Inflow for multiple conditions

When multiple cases are considered, the heterogeneous inflow conditions can be defined in two ways:

  1. Passing heterogeneous_inflow_config to the set method, with P points,
        and speedups of size n_findex X P
  2. Assigning heterogeneous_inflow_config_by_wd to the wind_data object
        used to drive FLORIS.  This object includes
        n_wd wind_directions, and speedups is of size n_wd X P.  When applied
          to set, the heterogeneous_inflow_config
        is automatically generated by using the nearest wind direction
        defined in heterogeneous_inflow_config_by_wd
        for each findex.

This example:

    1) Implements heterogeneous inflow for a 4 turbine layout using both of the above methods
    2) Compares the results of the two methods and shows that they are equivalent

"""


import matplotlib.pyplot as plt
import numpy as np

from floris import FlorisModel, TimeSeries


# Initialize FlorisModel
fmodel = FlorisModel("../inputs/gch.yaml")

# Change the layout to a 4 turbine layout in a box
fmodel.set(layout_x=[0, 0, 500.0, 500.0], layout_y=[0, 500.0, 0, 500.0])

# Define a TimeSeries object with 4 wind directions and constant wind speed
# and turbulence intensity

time_series = TimeSeries(
    wind_directions=np.array([269.0, 270.0, 271.0, 282.0]),
    wind_speeds=8.0,
    turbulence_intensities=0.06,
)

# Apply the time series to the FlorisModel
fmodel.set(wind_data=time_series)

# Define the x_locs to be used in the heterogeneous inflow configuration that form
# a box around the turbines
x_locs = [-500.0, -500.0, 1000.0, 1000.0]
y_locs = [-500.0, 1000.0, -500.0, 1000.0]

# Assume the speed-ups are defined such that they are the same 265-275 degrees and 275-285 degrees

# If defining heterogeneous_inflow_config directly, then the speedups are of size n_findex X P
# where the first 3 rows are identical, and the last row is different
speed_ups = [
    [1.0, 1.25, 1.0, 1.25],
    [1.0, 1.25, 1.0, 1.25],
    [1.0, 1.25, 1.0, 1.25],
    [1.0, 1.35, 1.0, 1.35],
]

heterogeneous_inflow_config = {
    "speed_multipliers": speed_ups,
    "x": x_locs,
    "y": y_locs,
}

# Set the heterogeneous inflow configuration
fmodel.set(heterogeneous_inflow_config=heterogeneous_inflow_config)

# Run the FLORIS simulation
fmodel.run()

# Get the power output of the turbines
turbine_powers = fmodel.get_turbine_powers() / 1000.0

# Now repeat using the wind_data object and heterogeneous_inflow_config_by_wd
# First, create the speedups for the two wind directions
speed_ups = [[1.0, 1.25, 1.0, 1.25], [1.0, 1.35, 1.0, 1.35]]

# Create the heterogeneous_inflow_config_by_wd dictionary
heterogeneous_inflow_config_by_wd = {
    "speed_multipliers": speed_ups,
    "x": x_locs,
    "y": y_locs,
    "wind_directions": [270.0, 280.0],
}

# Now create a new TimeSeries object including the heterogeneous_inflow_config_by_wd
time_series = TimeSeries(
    wind_directions=np.array([269.0, 270.0, 271.0, 282.0]),
    wind_speeds=8.0,
    turbulence_intensities=0.06,
    heterogeneous_inflow_config_by_wd=heterogeneous_inflow_config_by_wd,
)

# Apply the time series to the FlorisModel
fmodel.set(wind_data=time_series)

# Run the FLORIS simulation
fmodel.run()

# Get the power output of the turbines
turbine_powers_by_wd = fmodel.get_turbine_powers() / 1000.0

# Plot the results
wind_directions = fmodel.wind_directions
fig, axarr = plt.subplots(2, 2, sharex=True, sharey=True, figsize=(10, 10))
axarr = axarr.flatten()

for tindex in range(4):
    ax = axarr[tindex]
    ax.plot(wind_directions, turbine_powers[:, tindex], "ks-", label="Heterogeneous Inflow")
    ax.plot(
        wind_directions, turbine_powers_by_wd[:, tindex], ".--", label="Heterogeneous Inflow by WD"
    )
    ax.set_title(f"Turbine {tindex}")
    ax.set_xlabel("Wind Direction (deg)")
    ax.set_ylabel("Power (kW)")
    ax.legend()

plt.show()
import warnings
warnings.filterwarnings('ignore')